In [1]:
!pip install /kaggle/input/kerasapplications -q

In [2]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [3]:
!pip install /kaggle/input/efficientnet-100 -q
!pip install /kaggle/input/image-classifiers -q
!pip install /kaggle/input/segmentation-models -q

In [4]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import gc
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models

import efficientnet.tfkeras
import segmentation_models as sm
from sklearn.preprocessing import StandardScaler

Segmentation Models: using `tf.keras` framework.


# Helper functions

In [5]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    """
    get tf byte data from image files specified with path
    """
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0 # standardization
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    """
    increase the variation of images by ramdomly modifying
    """
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    # during first iteration, the input data will be cached in a directory or memory
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    # dataset operation are executed in order
    dset = tf.data.Dataset.from_tensor_slices(slices) # make dataset of paths
    dset = dset.map(decode_fn, num_parallel_calls=AUTO) # get image data from path
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset # prevent from overfitting
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

# Variables and configurations

In [6]:
COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = 8

Running on 1 replicas


In [7]:
target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged',
             'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

cvc_cols=['CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal']
non_cvc_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
    'NGT - Incompletely Imaged', 'NGT - Normal', 'Swan Ganz Catheter Present']

In [8]:
load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
sub_df = pd.read_csv(load_dir + 'sample_submission.csv')
test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

# Get the multi-labels
label_cols = sub_df.columns[1:]

# predct test data with EfficientNet_first

In [9]:
test_decoder = build_decoder(with_labels=False, target_size=(600, 600))
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

In [10]:
model_first_EfficientNet = tf.keras.models.load_model(
        "../input/ranzcr-chibamed/xhlulu/model.h5"
)
model_first_EfficientNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 19, 19, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 11)                28171     
Total params: 64,125,851
Trainable params: 63,815,131
Non-trainable params: 310,720
_________________________________________________________________


In [11]:
layer_name="global_average_pooling2d"
hidden_layer_model=models.Model(inputs=model_first_EfficientNet.input,outputs=model_first_EfficientNet.get_layer(layer_name).output)

model_first_EfficientNet_pred = hidden_layer_model.predict(dtest, verbose=1)

448/448 [==============================] - 302s 675ms/step


# CVC prediction

In [12]:
def build_gray_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=1)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=1)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode

In [13]:
test_decoder = build_gray_decoder(with_labels=True, target_size=(576, 576))
dtest = build_dataset(
    test_paths, sub_df["StudyInstanceUID"], bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=False, cache=False,
    decode_fn=test_decoder
)

In [14]:
def decode_str_tensors(tensors):
    return [tensor.numpy().decode() for tensor in tensors]

def create_pspnet():
    custom_objects={
        "iou_score":sm.metrics.IOUScore,
        "binary_crossentropy_plus_jaccard_loss":sm.losses.bce_jaccard_loss
    }
    model=tf.keras.models.load_model("../input/segmentation-ckpt/pspnet_ckpt", custom_objects=custom_objects)
    return model

pspnet=create_pspnet()

In [15]:
cvc_effnet=models.load_model("../input/efficientnetb7-cvc-tpu-train/cvc_effnet_ckpt")

In [16]:
cvc_pred=sub_df.copy()
for imgs,uids in tqdm(dtest):
    uids_decoded=decode_str_tensors(uids)
    preds=pspnet.predict(imgs)

    for uid,pred in zip(uids_decoded,preds):
        pred=pred.reshape(pred.shape[0],pred.shape[1])
        stacked_img=np.stack((pred,)*3,axis=-1)
        stacked_img[:,:,(1,2)]=0

        train_file_bytes=tf.io.read_file(f"../input/ranzcr-clip-catheter-line-classification/test/{uid}.jpg")
        train_img=tf.io.decode_jpeg(train_file_bytes,channels=3)
        train_img=tf.image.resize(train_img,(576,576))
        train_img/=255.0

        merged_img=train_img+stacked_img
        merged_img=merged_img.numpy()
        merged_img=np.where(merged_img>1,1,merged_img) # 加算によって通常の範囲を超えた値を制限する
        merged_img=np.expand_dims(merged_img,0)

        cvc_pred.loc[cvc_pred["StudyInstanceUID"]==uid,cvc_cols]=cvc_effnet.predict(merged_img)
    
    del preds
    gc.collect()

# other predictions

In [17]:
def compress_with_autoencoder(features):
    scaler=pickle.load(open("../input/ranzcr-chibamed/minmaxscaler_effnet_best.pickle","rb"))
    X=scaler.transform(features)

    autoencoder_dir="../input/ranzcr-chibamed/autoencoder_best/"
    with open(f"{autoencoder_dir}model.json","rt") as f:
        model_json=f.read()
    autoencoder=models.model_from_json(model_json)
    autoencoder.load_weights(f"{autoencoder_dir}ckpt")

    layer_name="dense_1"
    compressing_model=models.Model(inputs=autoencoder.input,outputs=autoencoder.get_layer(layer_name).output)

    ae_pred=compressing_model.predict(X)
    ae_pred_df=pd.DataFrame(ae_pred)

    return ae_pred_df

compressed=compress_with_autoencoder(model_first_EfficientNet_pred)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [18]:
valueless_columns=[3, 4, 6, 8, 9, 13, 14, 17, 18, 23, 27, 35, 36, 37, 38, 44, 45,
            47, 50, 51, 52, 57, 58, 61, 62, 67, 68, 72, 73, 74, 76, 85, 86, 87,
            90, 91, 92, 98, 99]
X_dropped=compressed.drop(columns=valueless_columns)
display(X_dropped)

,0,1,2,5,7,10,11,12,15,16,...,82,83,84,88,89,93,94,95,96,97
0,1.245343,1.953097,3.727363,1.631043,0.377156,8.707488,2.589135,5.754076,6.528375,1.472455,...,6.367021,2.353459,4.070107,1.414316,2.010886,4.112407,2.704958,1.375961,1.118705,3.303824
1,1.161254,2.004167,0.420246,0.636980,0.894705,0.880953,1.506587,1.709268,1.256729,1.056949,...,0.876100,0.906298,1.200082,0.000000,1.818242,1.025490,0.915230,2.537300,1.559568,0.886668
2,0.661021,1.667710,1.172590,0.466626,0.839070,1.993932,1.331750,1.252880,1.489470,1.065569,...,0.871546,0.931938,1.473553,0.000000,2.512747,2.184185,1.297052,2.554141,0.956345,1.670408
3,1.880408,1.591062,3.236990,1.040437,1.885902,3.164454,2.437915,2.255919,2.936540,0.761104,...,1.902773,2.808009,4.501772,0.492405,2.077940,3.170048,1.233046,1.937976,1.526051,1.150271
4,0.403669,1.197198,0.820887,0.153533,1.218934,1.314090,1.419266,1.951160,2.418241,1.114010,...,1.753063,1.582102,1.295339,0.129402,2.200849,1.118681,0.722933,2.863883,1.239728,1.943588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.828122,2.296063,0.799604,2.966467,1.378207,1.756912,1.690537,3.588608,1.873378,1.134539,...,0.508840,3.663941,1.752519,0.476301,2.345770,2.636401,1.392115,1.441780,1.479494,3.894493
3578,1.914372,4.314936,1.934809,0.384967,1.239418,1.248644,0.938784,4.112116,2.788666,3.359181,...,3.828135,1.113017,2.145099,0.000000,3.937252,0.899160,1.487881,1.707209,2.000372,3.221846
3579,1.365546,2.028641,1.192344,0.660027,1.053089,0.956426,1.506499,1.577883,1.432393,0.978273,...,0.850345,1.275596,0.657543,0.000000,2.260254,1.460700,0.845769,2.797733,1.437997,2.027253
3580,0.799241,3.426328,1.017505,0.633272,0.913083,0.911220,0.797396,4.758993,2.825311,2.908284,...,0.843065,1.087999,0.738445,0.000000,1.946380,1.425290,3.049413,1.014467,2.576335,2.774104


In [19]:
lgbm_pred=sub_df.copy()

for i,col_name in enumerate(target_cols):
    model=pickle.load(open(f"../input/ranzcr-chibamed/lgbm_effnet_best_dropped/model_{i}.pickle","rb"))
    pred=model.predict(X_dropped)
    lgbm_pred[col_name]=pd.Series(pred)

display(lgbm_pred)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.003496,0.020679,0.520220,0.026491,0.016140,0.606606,0.901664,0.495237,0.341768,0.239428,0.004556
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.003522,0.013875,0.458473,0.009022,0.017351,0.598931,0.811948,0.183727,0.370376,0.530517,0.000319
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.002809,0.029063,0.567953,0.007632,0.016976,0.625222,0.861252,0.434128,0.368440,0.433075,0.001436
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.003226,0.023195,0.460283,0.020271,0.017400,0.635916,0.783451,0.299054,0.348982,0.345856,0.000323
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.002875,0.018209,0.493709,0.015513,0.016806,0.601124,0.888170,0.246265,0.321850,0.421358,0.001416
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.81464483108873296584...,0.003544,0.015614,0.451930,0.021061,0.017656,0.551797,0.820474,0.212747,0.337328,0.479773,0.000320
3578,1.2.826.0.1.3680043.8.498.33579133018211530710...,0.003380,0.020874,0.592095,0.017457,0.018178,0.572838,0.848178,0.438536,0.303255,0.370932,0.000354
3579,1.2.826.0.1.3680043.8.498.61472811086105902907...,0.003096,0.021951,0.545655,0.010923,0.017208,0.632918,0.851060,0.346285,0.394178,0.376425,0.001751
3580,1.2.826.0.1.3680043.8.498.19434375795525494655...,0.002668,0.026838,0.522593,0.015126,0.018655,0.574025,0.897740,0.366576,0.266933,0.349872,0.000555


## Dense

In [20]:
def create_model(fold:int):
    dense_dir="../input/ranzcr-chibamed/effnet_best_dense/"
    with open(f"{dense_dir}model.json","rt") as f:
        model_json=f.read()
    dense_model=models.model_from_json(model_json)
    dense_model.load_weights(f"{dense_dir}ckpt_{fold}")
    return dense_model

dense_model_list=[create_model(n) for n in range(5)]

In [21]:
dense_pred_list=[pd.DataFrame(dense_model_list[n].predict(model_first_EfficientNet_pred),columns=target_cols) for n in range(5)]

dense_pred=sub_df.copy()

for col_name in target_cols:
    pred_col=pd.DataFrame()
    for n,pred in enumerate(dense_pred_list):
        pred_col[f"fold_{n}"]=pred[col_name]
    dense_pred[col_name]=pred_col.mean(axis=1)

display(dense_pred)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.015534,0.337775,0.441153,0.002469,0.002083,0.004972,0.988807,0.035746,0.196512,0.935037,0.999471
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0.000337,0.000133,0.000075,0.000073,0.000257,0.000051,0.000220,0.002397,0.007706,0.997755,0.000086
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0.000260,0.000280,0.000374,0.000234,0.000658,0.000128,0.000498,0.001375,0.137426,0.938434,0.000234
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.014480,0.165588,0.123099,0.046139,0.009520,0.961416,0.012046,0.154449,0.062081,0.927909,0.022233
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0.000405,0.000326,0.002101,0.000204,0.000320,0.000017,0.001014,0.002487,0.019238,0.991330,0.000110
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.81464483108873296584...,0.002774,0.003410,0.000626,0.003541,0.003946,0.925919,0.086923,0.012432,0.035649,0.981984,0.001358
3578,1.2.826.0.1.3680043.8.498.33579133018211530710...,0.000753,0.001759,0.000435,0.000674,0.000621,0.000433,0.003057,0.894700,0.179933,0.023034,0.000532
3579,1.2.826.0.1.3680043.8.498.61472811086105902907...,0.000168,0.000095,0.000172,0.000114,0.000289,0.000050,0.000274,0.000882,0.023471,0.981885,0.000071
3580,1.2.826.0.1.3680043.8.498.19434375795525494655...,0.000623,0.000960,0.000913,0.001176,0.001342,0.000581,0.001820,0.028140,0.941415,0.095463,0.000279


# make final submission

In [22]:
cvc_pred=cvc_pred[cvc_cols]
lgbm_pred=lgbm_pred[target_cols]
dense_pred=dense_pred[target_cols]

scaler=StandardScaler()

cvc_norm=pd.DataFrame(scaler.fit_transform(cvc_pred),columns=cvc_cols)
lgbm_norm=pd.DataFrame(scaler.fit_transform(lgbm_pred),columns=target_cols)
dense_norm=pd.DataFrame(scaler.fit_transform(dense_pred),columns=target_cols)

display(cvc_pred,lgbm_norm,dense_norm)

,CVC - Abnormal,CVC - Borderline,CVC - Normal
0,0.007127,0.057138,0.974337
1,0.002040,0.001199,0.997195
2,0.000296,0.073599,0.954633
3,0.007223,0.007493,0.999328
4,0.000699,0.009211,0.990315
...,...,...,...
3577,0.000307,0.004669,0.999457
3578,0.935856,0.225053,0.007561
3579,0.002893,0.032120,0.958069
3580,0.049845,0.975233,0.005405


,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,-0.212903,-0.360206,0.194933,2.472118,-1.385174,-0.244517,0.560274,1.441254,-0.092675,-2.279284,2.796779
1,-0.204190,-1.366454,-0.874591,-1.106875,-0.063912,-0.440692,-2.025135,-1.605871,0.465860,1.842093,-0.842740
2,-0.443876,0.879822,1.021721,-1.391708,-0.472772,0.231345,-0.604310,0.843496,0.428057,0.462452,0.116378
3,-0.303839,0.011923,-0.843236,1.197730,-0.009770,0.504697,-2.846375,-0.477768,0.048181,-0.772428,-0.839286
4,-0.421799,-0.725453,-0.264253,0.223098,-0.658160,-0.384641,0.171392,-0.994139,-0.481536,0.296571,0.099728
...,...,...,...,...,...,...,...,...,...,...,...
3577,-0.196722,-1.109252,-0.987919,1.359684,0.269419,-1.645517,-1.779451,-1.322005,-0.179355,1.123638,-0.841975
3578,-0.252018,-0.331374,1.439885,0.621344,0.839681,-1.107670,-0.981075,0.886618,-0.844552,-0.417390,-0.812719
3579,-0.347583,-0.172078,0.635501,-0.717291,-0.219891,0.428070,-0.898029,-0.015764,0.930535,-0.339626,0.387239
3580,-0.491394,0.550812,0.236042,0.143735,1.360200,-1.077337,0.447178,0.182719,-1.553683,-0.715567,-0.640374


,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,0.873933,3.563063,1.130990,-0.119356,-0.185823,-0.220304,3.941693,-0.264906,-0.257352,0.678873,8.936039
1,-0.098423,-0.197596,-0.368391,-0.219351,-0.231414,-0.248836,-0.318900,-0.429995,-0.875550,0.852646,-0.125859
2,-0.103363,-0.195967,-0.367373,-0.212650,-0.221411,-0.248394,-0.317704,-0.435051,-0.450813,0.688285,-0.124521
3,0.806542,1.645238,0.049813,1.703123,-0.000154,5.325602,-0.267932,0.322697,-0.697512,0.659125,0.074952
4,-0.094112,-0.195447,-0.361502,-0.213892,-0.229844,-0.249038,-0.315477,-0.429547,-0.837790,0.834844,-0.125646
...,...,...,...,...,...,...,...,...,...,...,...
3577,0.057511,-0.161102,-0.366519,-0.074646,-0.139320,5.119776,0.054770,-0.380316,-0.784058,0.808949,-0.114327
3578,-0.071802,-0.179486,-0.367165,-0.194286,-0.222326,-0.246622,-0.306673,3.987102,-0.311633,-1.848008,-0.121821
3579,-0.109284,-0.198021,-0.368061,-0.217661,-0.230616,-0.248846,-0.318668,-0.437493,-0.823932,0.808674,-0.125995
3580,-0.080155,-0.188395,-0.365541,-0.173336,-0.204336,-0.245768,-0.312003,-0.302560,2.181655,-1.647329,-0.124110


In [23]:
best_params={
    'ETT - Abnormal':[0.0],
    'ETT - Borderline':[0.0],
    'ETT - Normal':[0.0],
    'NGT - Abnormal':[0.0],
    'NGT - Borderline':[0.01],
    'NGT - Incompletely Imaged':[0.01],
    'NGT - Normal':[0.0],
    'CVC - Abnormal':[0.0,0.1],
    'CVC - Borderline':[0.0,0.11],
    'CVC - Normal':[0.0,0.16],
    'Swan Ganz Catheter Present':[0.0]
}

In [24]:
def get_ensemble_pred_2(col_name):
    p=best_params[col_name][0]
    ensemble_pred=p*lgbm_norm[col_name]+(1-p)*dense_norm[col_name]
    return ensemble_pred

def get_ensemble_pred_3(col_name):
    p=best_params[col_name][0]
    q=best_params[col_name][1]
    ensemble_pred=p*lgbm_norm[col_name]+q*dense_norm[col_name]+(1-p-q)*cvc_norm[col_name]
    return ensemble_pred

In [25]:
submission=sub_df.copy()

for col_name in non_cvc_cols:
    submission[col_name]=pd.Series(get_ensemble_pred_2(col_name))

for col_name in cvc_cols:
    submission[col_name]=pd.Series(get_ensemble_pred_3(col_name))

submission.to_csv("submission.csv",index=False)
display(submission)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0.873933,3.563063,1.130990,-0.119356,-0.197817,-0.220546,3.941693,-0.357501,-0.577124,0.702457,8.936039
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,-0.098423,-0.197596,-0.368391,-0.219351,-0.229739,-0.250755,-0.318900,-0.394423,-0.788153,0.779742,-0.125859
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,-0.103363,-0.195967,-0.367373,-0.212650,-0.223924,-0.243597,-0.317704,-0.401928,-0.556318,0.661308,-0.124521
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0.806542,1.645238,0.049813,1.703123,-0.000250,5.277393,-0.267932,-0.298355,-0.752476,0.753396,0.074952
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,-0.094112,-0.195447,-0.361502,-0.213892,-0.234127,-0.250394,-0.315477,-0.399760,-0.763513,0.762001,-0.125646
...,...,...,...,...,...,...,...,...,...,...,...,...
3577,1.2.826.0.1.3680043.8.498.81464483108873296584...,0.057511,-0.161102,-0.366519,-0.074646,-0.135233,5.052124,0.054770,-0.396410,-0.769216,0.777648,-0.114327
3578,1.2.826.0.1.3680043.8.498.33579133018211530710...,-0.071802,-0.179486,-0.367165,-0.194286,-0.211706,-0.255232,-0.306673,3.794398,-0.153769,-1.794694,-0.121821
3579,1.2.826.0.1.3680043.8.498.61472811086105902907...,-0.109284,-0.198021,-0.368061,-0.217661,-0.230509,-0.242077,-0.318668,-0.391748,-0.703416,0.688008,-0.125995
3580,1.2.826.0.1.3680043.8.498.19434375795525494655...,-0.080155,-0.188395,-0.365541,-0.173336,-0.188691,-0.254083,-0.312003,-0.189854,2.038559,-1.767253,-0.124110
